---
title: Network Matrices and Pandapower Interface
toc: true
number-sections: true
mainfont: Arial
format:
  html: default
jupyter: python3
---

In [1]:
#| echo: false
#| output: false
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r"..\..\src")

import matplotlib
matplotlib.rcParams.update({"axes.grid" : True}) # add grid by default

This tutorial shows how to access network matrices (such as the *admittance matrix* of the *load flow Jacobian*). *PowerFactory* does not allow this access directly, so the dataset is first exported to the [*pandapower* format](https://pandapower.readthedocs.io/en/stable/index.html). The matrices can then be extracted from this format.

You can learn more from the *pandapower* documentation:

- [*PowerFactory* to *pandapower* converter](https://pandapower.readthedocs.io/en/stable/converter/powerfactory.html)
- [Tutorial on network matrices](https://github.com/e2nIEE/pandapower/blob/develop/tutorials/internal_datastructure.ipynb)

::: {.callout-warning}
Note that not all networks elements/components are supported by pandapower (see the [list of supported components](https://pandapower.readthedocs.io/en/stable/converter/powerfactory.html)) and the functionality shown in this tutorial may not be applicable to some *PowerFactory* datasets.

:::

## Imports and Project Activation
First we import some packages and activate the *PowerFactory* project from which we want to export the dataset (the *IEEE 39 bus system* example) and get the network matrices.

In [2]:
import os

import numpy as np
from icecream import ic
import pandas as pd
import pandapower as pp

from powfacpy.base.active_project import ActiveProject
from powfacpy.pf_classes.protocols import PFApp

# If you use IPython/Jupyter:
import sys

sys.path.append(
    r"C:\Program Files\DIgSILENT\PowerFactory 2025 SP3\Python\3.13"
)  # you may use a different directory
# Get the PF app
import powerfactory

app = powerfactory.GetApplication()

pf_app: PFApp = powerfactory.GetApplication()
act_prj = ActiveProject(pf_app)
act_prj.app.Show()
act_prj.app.ActivateProject(
    "powfacpy\\39_bus_new_england_copy_where_tests_run"
)  # You may change the project path.
act_prj.activate_study_case("Study Cases\\2.1 Simulation Fault Bus 16 Stable")

<powerfactory.DataObject <l1>\seberlein.IntUser\powfacpy\39_bus_new_england_copy_where_tests_run.IntPrj\Study Cases.IntPrjfolder\2.1 Simulation Fault Bus 16 Stable.IntCase</l1>>

## Export Dataset to Pandapower
The dataset can be exported to *pandapower* format using the `pf_project_to_pandapower` function. It requires the *PowerFactory* app and the path to the project as inpit arguments.

In [3]:
from powfacpy.applications.pandapower_interface import PandapowerInterface

try:
    act_prj.app.Hide()
    ppi = PandapowerInterface(app)
    net = ppi.pf_project_to_pandapower()
finally:
    act_prj.app.Show()
net

This pandapower network includes the following parameter tables:
   - bus (39 elements)
   - load (19 elements)
   - gen (9 elements)
   - switch (92 elements)
   - ext_grid (1 element)
   - line (34 elements)
   - trafo (12 elements)
 and the following results tables:
   - res_bus (39 elements)
   - res_line (34 elements)
   - res_trafo (12 elements)
   - res_ext_grid (1 element)
   - res_load (19 elements)
   - res_gen (9 elements)
   - res_switch (92 elements)

`net` gives an overview of the exported network components and result tables (*pandapower* result format). *PandaPower* is based on *pandas* dataframes. You can query for example the data of the buses using:

In [4]:
net.bus

name  vn_kv type  zone  in_service  \
0   Bus 08  345.0    b  Grid        True   
1   Bus 07  345.0    b  Grid        True   
2   Bus 05  345.0    b  Grid        True   
3   Bus 04  345.0    b  Grid        True   
4   Bus 06  345.0    b  Grid        True   
5   Bus 31   16.5    b  Grid        True   
6   Bus 11  345.0    b  Grid        True   
7   Bus 12  138.0    b  Grid        True   
8   Bus 10  345.0    b  Grid        True   
9   Bus 32   16.5    b  Grid        True   
10  Bus 13  345.0    b  Grid        True   
11  Bus 14  345.0    b  Grid        True   
12  Bus 15  345.0    b  Grid        True   
13  Bus 37   16.5    b  Grid        True   
14  Bus 17  345.0    b  Grid        True   
15  Bus 27  345.0    b  Grid        True   
16  Bus 18  345.0    b  Grid        True   
17  Bus 03  345.0    b  Grid        True   
18  Bus 26  345.0    b  Grid        True   
19  Bus 28  345.0    b  Grid        True   
20  Bus 25  345.0    b  Grid        True   
21  Bus 29  345.0    b  Grid        True   
22  Bus 38   16.5    b  Grid        True   
23  Bus 30   16.5    b  Grid        True   
24  Bus 02  345.0    b  Grid        True   
25  Bus 01  345.0    b  Grid        True   
26  Bus 39  345.0    b  Grid        True   
27  Bus 09  345.0    b  Grid        True   
28  Bus 36   16.5    b  Grid        True   
29  Bus 23  345.0    b  Grid        True   
30  Bus 22  345.0    b  Grid        True   
31  Bus 35   16.5    b  Grid        True   
32  Bus 19  345.0    b  Grid        True   
33  Bus 20  230.0    b  Grid        True   
34  Bus 33   16.5    b  Grid        True   
35  Bus 24  345.0    b  Grid        True   
36  Bus 34   16.5    b  Grid        True   
37  Bus 21  345.0    b  Grid        True   
38  Bus 16  345.0    b  Grid        True   

                                          geo description substat folder_id  \
0   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
1   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
2   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
3   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
4   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
5   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
6   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
7   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
8   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
9   {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
10  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
11  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
12  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
13  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
14  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
15  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
16  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
17  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
18  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
19  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
20  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
21  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
22  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
23  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
24  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
25  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
26  {"coordinates":[0.0,0.0], "type":"Point"}        Grid              Grid   
27  {"coordinates":[0.0,0.0], "ty

Note that the `name` column is filled with the `loc_name` attributes of the *PowerFactory* objects. `name` can be used as a mapping to the respective *PowerFactory* object only if the `loc_name` of all objects of a certain class are unique (e.g. no `ElmTerm` instances have the same `loc_name`). The `Database` interface can be used to enumerate equivalent names of calculation relevant objects in the *PowerFactory* database. 

In [5]:
from powfacpy.applications.database import Database

try:
    act_prj.app.Hide()
    dbi = Database(app)
    dbi.make_loc_name_unique()
    net = ppi.pf_project_to_pandapower()
finally:
    act_prj.app.Show()

To test whether the exported dataset is feasible, we can run a power flow using *pandapower*:

In [6]:
pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


The following shows how to access the results for a certain bus.

In [7]:
bus_name = "Bus 05"
index_of_element = net.bus[net.bus["name"] == bus_name].index[0]
net.res_bus.iloc[index_of_element]

vm_pu        1.005311
va_degree   -8.611863
p_mw         0.000000
q_mvar       0.000000
Name: 2, dtype: float64

## Network Matrices
### Admittance Matrix
We can no extract the *Ybus* (i.e admittance) matrix. By default, the returned *numpy* matrix is sparse, so we create a dense matrix for printing.

In [8]:
Yb = ppi.get_Ybus_matrix(net)

print(Yb)

[[ 26.84540295-332.03038228j -18.76172734+215.75984428j
   -6.34517667 +88.83248222j ...   0.          +0.j
    0.          +0.j           0.          +0.j        ]
 [-18.76172734+215.75984428j  25.82055168-323.99513559j
    0.          +0.j         ...   0.          +0.j
    0.          +0.j           0.          +0.j        ]
 [ -6.34517667 +88.83248222j   0.          +0.j
   40.62074723-549.00640737j ...   0.          +0.j
    0.          +0.j           0.          +0.j        ]
 ...
 [  0.          +0.j           0.          +0.j
    0.          +0.j         ...   2.74613638 -54.92271706j
    0.          +0.j           0.          +0.j        ]
 [  0.          +0.j           0.          +0.j
    0.          +0.j         ...   0.          +0.j
    8.44256311-145.01095393j  -4.37421432 +73.8148601j ]
 [  0.          +0.j           0.          +0.j
    0.          +0.j         ...   0.          +0.j
   -4.37421432 +73.8148601j   36.02583738-510.89377228j]]


The base apparent power is `net._ppc["baseMVA"]`.

In [9]:
net._ppc["baseMVA"]

100.0

This does not provide any information about the corresponding nodes of the rows and columns. The function `get_Ybus_frame` provides this information.

In [10]:
df = ppi.get_Ybus_frame(net)
df

Bus 08                 Bus 07                 Bus 05  \
Bus 08  26.845403-332.030382j -18.761727+215.759844j  -6.345177+ 88.832482j   
Bus 07 -18.761727+215.759844j  25.820552-323.995136j   0.000000+  0.000000j   
Bus 05  -6.345177+ 88.832482j   0.000000+  0.000000j  40.620747-549.006407j   
Bus 04   0.000000+  0.000000j   0.000000+  0.000000j  -4.863813+ 77.821008j   
Bus 06   0.000000+  0.000000j  -7.058824+108.235291j -29.411758+382.352917j   
Bus 31   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 11   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 12   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 10   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 32   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 13   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 14   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 15   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 37   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 17   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 27   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 18   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 03   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 26   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 28   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 25   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 29   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 38   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 30   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 02   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 01   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 39   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 09  -1.738499+ 27.438056j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 36   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 23   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 22   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 35   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 19   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 20   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 33   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 24   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 34   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 21   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   
Bus 16   0.000000+  0.000000j   0.000000+  0.000000j   0.000000+  0.000000j   

                       Bus 04                 Bus 06               Bus 31  \
Bus 08   0.000000+  0.000000j   0.000000+  0.000000j  0.000000+ 0.000000j   
Bus 07   0.000000+  0.000000j  -7.058824+108.235291j  0.000000+ 0.000000j   
Bus 05  -4.863813+ 77.821008j -29.411758+382.352917j  0.000000+ 0.000000j   
Bus 04  12.507556-201.817511j   0.000000+  0.000000j  0.000000+ 0.000000j   
Bus 06   0.000000+  0.000000j  46.805737-649.040329j  0.000000+37.383178j   
Bus 31   0.000000+  0.000000j   0.000000+ 37.383178j  0.000000-37.383178j   
Bus 11   0.000000+  0.000000j -10.335155+121.068943j  0.000000+ 0.000000j   
Bus 12   0.000000+  0.000000j   0.000000+  0.000000j  0.000000+ 0.000000j   
Bus 10   0.000000+  0.000000j   0.000000+  0.000000j  0.000000+ 0.000000j   
Bus 32   0.000000+  0.000000j   0.000000+  0.000000j  0.000000+ 0.000000j   
Bus 13   0.0000

The names in the labels correspond to the `loc_name` attributes of the *PowerFactory* objects. Provided that the names are unique, we can also replace them with the actual *PowerFactory* objects (this can be more convenient to query data, but printing the frame is ugly because the full path is used in the labels) using the `PandasInterface` of *powfacpy*.

In [11]:
from powfacpy.applications.pandas_interface import PandasInterface

pdi = PandasInterface(app)

df = pdi.replace_loc_name_with_pf_objects_in_labels(
    df, "ElmTerm", index_and_column_labels_are_equal=True
)
df

<l3>\seberlein.IntUser\powfacpy\39_bus_new_england_copy_where_tests_run.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\Bus 08.ElmTerm</l3>  \
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                              26.845403-332.030382j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                             -18.761727+215.759844j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                              -6.345177+ 88.832482j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j                                                                                                                     
<l3>\seberlein.IntUser\powfacpy\39_bus_new_engl...                               0.000000+  0.000000j         

### Connectivity (Adjacency) Matrix
The connectivity matrix (also called adjacency matrix) is a symmetric matrix that has entries $1$ (or True) for connected nodes and $0$ (or False) for nodes that are not connected. 

In [12]:
ppi.get_connectivity_frame(net)

Bus 08  Bus 07  Bus 05  Bus 04  Bus 06  Bus 31  Bus 11  Bus 12  \
Bus 08       1       1       1       0       0       0       0       0   
Bus 07       1       1       0       0       1       0       0       0   
Bus 05       1       0       1       1       1       0       0       0   
Bus 04       0       0       1       1       0       0       0       0   
Bus 06       0       1       1       0       1       1       1       0   
Bus 31       0       0       0       0       1       1       0       0   
Bus 11       0       0       0       0       1       0       1       1   
Bus 12       0       0       0       0       0       0       1       1   
Bus 10       0       0       0       0       0       0       1       0   
Bus 32       0       0       0       0       0       0       0       0   
Bus 13       0       0       0       0       0       0       0       1   
Bus 14       0       0       0       1       0       0       0       0   
Bus 15       0       0       0       0       0       0       0       0   
Bus 37       0       0       0       0       0       0       0       0   
Bus 17       0       0       0       0       0       0       0       0   
Bus 27       0       0       0       0       0       0       0       0   
Bus 18       0       0       0       0       0       0       0       0   
Bus 03       0       0       0       1       0       0       0       0   
Bus 26       0       0       0       0       0       0       0       0   
Bus 28       0       0       0       0       0       0       0       0   
Bus 25       0       0       0       0       0       0       0       0   
Bus 29       0       0       0       0       0       0       0       0   
Bus 38       0       0       0       0       0       0       0       0   
Bus 30       0       0       0       0       0       0       0       0   
Bus 02       0       0       0       0       0       0       0       0   
Bus 01       0       0       0       0       0       0       0       0   
Bus 39       0       0       0       0       0       0       0       0   
Bus 09       1       0       0       0       0       0       0       0   
Bus 36       0       0       0       0       0       0       0       0   
Bus 23       0       0       0       0       0       0       0       0   
Bus 22       0       0       0       0       0       0       0       0   
Bus 35       0       0       0       0       0       0       0       0   
Bus 19       0       0       0       0       0       0       0       0   
Bus 20       0       0       0       0       0       0       0       0   
Bus 33       0       0       0       0       0       0       0       0   
Bus 24       0       0       0       0       0       0       0       0   
Bus 34       0       0       0       0       0       0       0       0   
Bus 21       0       0       0       0       0       0       0       0   
Bus 16       0       0       0       0       0       0       0       0   

        Bus 10  Bus 32  ...  Bus 23  Bus 22  Bus 35  Bus 19  Bus 20  Bus 33  \
Bus 08       0       0  ...       0       0       0       0       0       0   
Bus 07       0       0  ...       0       0       0       0       0       0   
Bus 05       0       0  ...       0       0       0       0       0       0   
Bus 04       0       0  ...       0       0       0       0       0       0   
Bus 06       0       0  ...       0       0       0       0       0       0   
Bus 31       0       0  ...       0       0       0       0       0       0   
Bus 11       1       0  ...       0       0       0       0       0       0   
Bus 12       0       0  ...       0       0       0       0       0       0   
Bus 10       1       1  ...       0       0       0       0       0       0   
Bus 32       1       1  ...       0       0       0       0       0       0   
Bus 13       1       0  ...       0       0       0       0       0       0   
Bus 14       0       0  ...       0       0       0       0       0       0   
Bus 15       0      

To get boolean values instead of zero and one, use `ppi.get_connectivity_frame(net, boolean=True)`.

## Load Flow Jacobian
The load flow Jacobian matrix can be exported as well.

In [13]:
net._ppc["internal"]["J"].todense()

matrix([[  46.27299714,    0.        ,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        [   0.        ,   45.49863821,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,   67.13959559, ...,    0.        ,
            0.        ,    0.        ],
        ...,
        [   0.        ,    0.        ,    0.        , ...,  205.55025042,
            0.        , -175.3811615 ],
        [   0.        ,    0.        ,    0.        , ...,    0.        ,
          148.2388436 ,  -76.28193549],
        [   0.        ,    0.        ,    0.        , ..., -174.4079501 ,
          -75.90390706,  526.3356963 ]], shape=(67, 67))

An explanation of the Jacobian matrix entries is provided in [the pandapower tutorial](https://github.com/e2nIEE/pandapower/blob/develop/tutorials/internal_datastructure.ipynb) that was mentioned above already.

## Format Comparison {#sec-format-comparison}
The dataformats of *pandapower* and *PowerFactory* are not fully compatible. An overview and functionality to rudimentary validate the exported data is presented in the following.

Remember the exported dataset in *pandapower* format:

In [14]:
net

This pandapower network includes the following parameter tables:
   - bus (39 elements)
   - load (19 elements)
   - gen (9 elements)
   - switch (92 elements)
   - ext_grid (1 element)
   - line (34 elements)
   - trafo (12 elements)
 and the following results tables:
   - res_bus (39 elements)
   - res_line (34 elements)
   - res_trafo (12 elements)
   - res_ext_grid (1 element)
   - res_load (19 elements)
   - res_gen (9 elements)
   - res_switch (92 elements)

There are a few things to consider here (not everything is relevant for this example):

- *bus* considers all terminals, even those that are not energized
- *gen* are *PV*-controlled generators (active power and voltage)
- *sgen* are *PQ*-controlled generators
- *ext_grid* are generators/external grids which act as slack buses
- *switch* considers only breakers/switches for branches, i.e. between buses or between buses and branch elements like lines and transformers (i.e. switches connecting generators are not considered for example)


## Validation
### Using *pandapower*
*pandapower* has a function `validate_pf_conversion` (due to a [bug in *pandapower*](https://github.com/e2nIEE/pandapower/issues/2363) this function fails if load flow results are initially present) which compares load flow results and returns the differences. 

In [15]:
from pandapower.converter.powerfactory.validate import validate_pf_conversion

try:
    act_prj.app.Hide()
    net = ppi.pf_project_to_pandapower()
    all_diffs = validate_pf_conversion(net)
finally:
    act_prj.app.Show()
all_diffs

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


{'gen_p_diff_is':            diff  p_mw_pp  p_mw_pf
 0  5.388387e-09    250.0    250.0
 1  4.945377e-09    540.0    540.0
 2  4.474373e-09   1000.0   1000.0
 3  4.093181e-09    650.0    650.0
 4  1.336514e-08    632.0    632.0
 5  5.883180e-09    508.0    508.0
 6  6.877826e-09    650.0    650.0
 7  5.005631e-09    830.0    830.0
 8  1.286480e-09    560.0    560.0,
 'gen_q_diff_is':            diff   q_mvar_pp   q_mvar_pf
 0 -1.990342e-06  146.158180  146.158178
 1  3.744761e-06    0.439923    0.439926
 2  3.910830e-07   88.281414   88.281415
 3  1.046724e-06  205.144370  205.144371
 4  2.652365e-06  109.905979  109.905981
 5 -3.168794e-06  165.763652  165.763649
 6  4.919206e-07  212.411826  212.411827
 7  1.547455e-06   22.841623   22.841625
 8  1.132399e-06  101.175009  101.175010,
 'diff_vm':             diff  vm_pu_pp  vm_pu_pf
 0   3.273640e-10  0.996020  0.996020
 1   3.792664e-10  0.997001  0.997001
 2   4.445253e-10  1.005311  1.005311
 3   3.405909e-10  1.003863  1.003863
 4 

### Using *powfacpy*
*powfacpy* offers a rudimentary validation of the exported dataset focusing only on data relevant for the admittance matrix. It is assumed that the currently active project in *PowerFactory* was used for the export. We can check the difference between both datasets by using `get_difference_between_pf_and_pandapower_dataset`. 

We artificially set (and later undo) a deviation in the *pandapower* dataset here. A *Dataframe* with the divergent parameters is returned.

In [16]:
from pandas import DataFrame

try:
    pf_app.Hide()
    net["bus"].loc[0, "vn_kv"] = net["bus"].loc[0, "vn_kv"] + 1

    divergent_parameters = ppi.get_difference_between_pf_and_pandapower_dataset(net)
    net["bus"].loc[0, "vn_kv"] = net["bus"].loc[0, "vn_kv"] - 1

finally:
    pf_app.Show()

divergent_parameters

name pf_class pp_parameter  pf_value  pp_value
0  Bus 08  ElmTerm        vn_kv     345.0     346.0

`get_difference_between_pf_and_pandapower_dataset` assumes default settings for the units in the *PowerFactory* project (see *Settings\Units*).

As mentioned, only a subset of classes and parameters is validated. The following mapping is used:

In [17]:
ppi.get_pandapower_2_pf_class_mapping()

{'bus': ['ElmTerm'],
 'line': ['ElmLne'],
 'trafo': ['ElmTr2', 'ElmTr3'],
 'impedance': ['ElmZpu', 'ElmSind']}

The parameter mapping is defined in `get_pandapower_2_pf_parameter_mapping`. It is best understood by looking at the source code and we use *python's* `inspect` module to print the method. The mapping is either between the *pandapower parameter strings or between a *pandapower* parameter string and and a callable with the *PowerFactory* object as argument and returning the parameter value (used for example to get parameters from the *type* object). 

In [18]:
import inspect

print(f"{inspect.getsource(ppi.get_pandapower_2_pf_parameter_mapping)}")

    def get_pandapower_2_pf_parameter_mapping(
        self,
    ) -> dict[str, dict[str, str | Callable]]:
        """Get parameter mapping between pandapower and PowerFactory.

        Only a small subset of all the parameters is currently implemented.

        Returns:
            dict[str, dict[str, str | Callable]]: For each pf class, a mapping between the pandapower and the pf parameter is provided. The pf parameter can be a string or a callable (input is the pf object; used for example to access a parameter of the type).
        """
        return {
            "ElmTerm": {
                "vn_kv": "uknom",
            },
            "ElmLne": {
                "length_km": "dline",
                "parallel": "nlnum",
                "r_ohm_per_km": lambda x: x.typ_id.rline,
                "x_ohm_per_km": lambda x: x.typ_id.xline,
                "c_nf_per_km": lambda x: x.typ_id.cline
                * 1000  # from micro to nano
                * x.typ_id.frnom  # nominal fre